In [15]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV ,KFold
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from sklearn.model_selection import cross_val_score

In [16]:
from sklearn.calibration import LabelEncoder
df = pd.read_csv('Data/colon-dataset-processed.csv')
le = LabelEncoder()
for column in df.columns:
    df[column] = le.fit_transform(df[column])
df

,Class,Age,p16540_C/C,p16540_G/C,p16540_G/G,p16580_C/C,p16580_C/T,p16580_T/T,mdm2_G/G,mdm2_G/T,mdm2_T/T,GAL3_A/A,GAL3_C/A,GAL3_C/C,TIM1_C/C,TIM1_G/C,TIM1_G/G
0,1,23,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0
1,1,23,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0
2,1,23,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0
3,1,12,0,1,0,1,0,0,0,0,1,0,1,0,1,0,0
4,1,23,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0,46,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1
111,0,39,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1
112,0,27,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0
113,0,40,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1


In [17]:
data=df.drop(['Class'],axis=1)
target=df['Class']
print('Original dataset distribution %s' % Counter(target))
random_state=42

Original dataset distribution Counter({0: 65, 1: 50})


In [ ]:
%pip install xgboost

In [33]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


model_params = {
    
    'AdaBoostClassifier' : {
        'model': AdaBoostClassifier(),
        'params': {
            'n_estimators': [50, 100, 200], 
            'learning_rate': [0.01, 0.1, 1]
        }
    } ,
    
    'DecisionTreeClassifier' : {
        'model': DecisionTreeClassifier(random_state=42),
        'params': {
            'max_depth': [2, 3, 5, 10, 20],
            'min_samples_leaf': [5, 10, 20, 50, 100],
            'criterion': ["gini", "entropy"]
        }
    },
    
    'ExtraTreesClassifier' : {
        'model': ExtraTreesClassifier(),
        'params': {
        'n_estimators': [50, 100, 200],
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 4, 6],
        'min_samples_leaf': [1, 2, 3],
        'max_features': ['auto', 'sqrt', 'log2']
        }
    },
    
    'HistGradientBoostingClassifier' : {
        'model': HistGradientBoostingClassifier(),
        'params': {
            'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
            'max_iter': [50, 100, 200],
            'max_leaf_nodes': [15, 31, 63, 127],
            'min_samples_leaf': [5, 10, 20],
            'l2_regularization': [0.0, 0.1, 1.0]
        }
    },
    
    'KNeighborsClassifier' : {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 10, 15],  # Usually, small numbers are chosen
            'weights': ['uniform', 'distance'],  # 'distance' weights points by the inverse of their distance
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # 'auto' will decide the most appropriate algorithm based on the values passed to fit method
            'leaf_size': [10, 30, 50],  # Can affect the speed of the construction and query, as well as the memory required to store the tree
            'p': [1, 2],  # 1 for manhattan_distance, 2 for euclidean_distance
            'metric': ['minkowski', 'euclidean', 'manhattan']  # Minkowski is a generalization that includes euclidean and manhattan
        }
    },
    
    
}

In [34]:
scores = []
n_splits = 15
cv=KFold(n_splits=n_splits, random_state=random_state, shuffle=True)
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False,n_jobs=-1,error_score=0)
    clf.fit(data, target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it 

In [35]:
model_params = {
    # 'LGBMClassifier' : {
    #     'model': LGBMClassifier(),
    #     'params': {
    #         'boosting_type': ['gbdt'],  # or you can choose 'dart' or 'goss'
    #         'num_leaves': [31, 50],  # Larger values can lead to overfitting
    #         'max_depth': [5, 10],  # -1 means no limit
    #         'learning_rate': [0.05, 0.1],  # Smaller values make learning more robust
    #         'n_estimators': [100, 200],  # Number of boosted trees to fit
    #         'min_child_samples': [20, 30],  # Minimum number of data needed in a leaf
    #         'subsample': [0.8, 1.0],  # Subsample ratio of the training instance
    #         'colsample_bytree': [0.8, 1.0] 
    #     }
    # },
    
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'penalty': ['l1', 'l2', 'elasticnet', 'none'],
            'C': [0.001, 0.01, 0.1, 1, 10, 100],
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
            'max_iter': [100, 200, 300, 400, 500]
        }
    },
    
    'RandomForestClassifier' : {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': [25, 50, 100, 150], 
            'max_features': ['sqrt', 'log2', None], 
            'max_depth': [3, 6, 9], 
            'max_leaf_nodes': [3, 6, 9]
        }
    },
    
    'XGBClassifier' : {
        'model': XGBClassifier(),
        'params': {
            'max_depth': [3, 5, 7], 
            'learning_rate': [0.01, 0.1, 0.2], 
            'n_estimators': [100, 200, 300]
        }
    },
    
    'svm': {
        'model':SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
}

In [36]:
cv=KFold(n_splits=n_splits, random_state=random_state, shuffle=True)
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False,n_jobs=-1,error_score=0)
    clf.fit(data, target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn

In [37]:
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,AdaBoostClassifier,0.860714,"{'learning_rate': 1, 'n_estimators': 50}"
1,DecisionTreeClassifier,0.851190,"{'criterion': 'gini', 'max_depth': 2, 'min_sam..."
2,ExtraTreesClassifier,0.800000,"{'criterion': 'entropy', 'max_depth': None, 'm..."
3,HistGradientBoostingClassifier,0.859524,"{'l2_regularization': 0.1, 'learning_rate': 0...."
4,KNeighborsClassifier,0.866667,"{'algorithm': 'auto', 'leaf_size': 10, 'metric..."
5,logistic_regression,0.841667,"{'C': 0.001, 'max_iter': 100, 'penalty': 'l2',..."
6,RandomForestClassifier,0.859524,"{'max_depth': 6, 'max_features': 'sqrt', 'max_..."
7,XGBClassifier,0.851190,"{'learning_rate': 0.01, 'max_depth': 5, 'n_est..."
8,svm,0.858333,"{'C': 1, 'kernel': 'linear'}"
